# Método das Diferenças Finitas
Rafael Brusiquesi Martins - 11711EQU014

Neste notebook será feita uma implementação e teste do método das diferenças finitas para o caso bidimensional da equação simplificada da transferência de energia térmica, por mecanismo difusivo.

In [98]:
import numpy as np
from copy import deepcopy
import matplotlib .pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

Definição da classe FiniteDifferences que recebe um grid sujeito a uma condição de contorno inicial, e aplica o método das diferenças finitas em cada elemento da matriz para cada iteração temporal.

In [99]:
class FiniteDifferences:
    def __init__(self, grid, boundaries):
        self.grid = grid
        self.boundaries = boundaries
        
        self.delta_x = 1
        self.delta_y = 1

    def get_y_diff_points(self, i, j):    
        if i - 1 < 0:
            y_anterior = self.boundaries['top'][j]
            y_posterior = self.grid[i+1][j]
        
        elif i + 1 >= self.grid.shape[0]:
            y_anterior = self.grid[i-1][j]
            y_posterior = self.boundaries['down'][j]
        
        else:
            y_anterior = self.grid[i-1][j]
            y_posterior = self.grid[i+1][j]
        
        return [y_anterior, y_posterior]

    def get_x_diff_points(self, i, j):    
        if j - 1 < 0:
            x_anterior = self.boundaries['left'][i]
            x_posterior = self.grid[i][j+1]
        
        elif j + 1 >= self.grid.shape[1]:
            x_anterior = self.grid[i][j-1]
            x_posterior = self.boundaries['right'][i]
        
        else:
            x_anterior = self.grid[i][j-1]
            x_posterior = self.grid[i][j+1]
        
        return [x_anterior, x_posterior]
    
    def D2Dx(self, i, j):
        (x_anterior, x_posterior) = self.get_x_diff_points(i, j)
        x_central = self.grid[i][j]
        
        return (x_posterior - 2*x_central + x_anterior) / (self.delta_x**2)

    def D2Dy(self, i, j):
        (y_anterior, y_posterior) = self.get_y_diff_points(i, j)
        y_central = self.grid[i][j]
        
        return (y_posterior - 2*y_central + y_anterior) / (self.delta_y**2)

    def forward_time(self, delta_t):
        new_grid = deepcopy(self.grid)
        for i in range(self.grid.shape[0]):
            for j in range(self.grid.shape[1]):
                new_grid[i][j] = self.grid[i][j] + delta_t * (self.D2Dx(i, j) + self.D2Dy(i, j)) # Equação da difusão
        
        self.grid = new_grid
        return self.grid

Definindo a malha de condições iniciais e de contorno e realizando a simulação.

In [172]:
grid = np.ones(shape=(75,75))
boundaries = {
    'top'  : [1]*grid.shape[1],
    'left' : [1]*grid.shape[0],
    'down' : [1]*grid.shape[1],
    'right': [1]*grid.shape[0],
}

for i in range(3):
    boundaries['left'][i] = 0

plotlist = []
FD = FiniteDifferences(grid, boundaries)
for i in range(5000):
    if i % 50 == 0:
        FD.grid[np.random.randint(1, 74)][np.random.randint(1, 74)] = np.random.uniform(low=30, high=200)
    result = FD.forward_time(0.2)
    plotlist.append(result)

Criando uma animação do processo.

In [173]:
def getImageFromList(x):
    return plotlist[x]

fig = plt.figure(figsize=(10, 10))
ims = []
for i in range(len(plotlist)):
    im = plt.imshow(getImageFromList(i), animated=True, vmin=1, vmax=5)
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True, repeat_delay=1000)
plt.close()

HTML(ani.to_html5_video())

Assim, pode-se perceber que o método das diferenças finitas representa uma boa alternativa para a computação de resultados discretizados para equações diferenciais parciais, que por sua vez, modelam e representam uma grande variedade de fenômenos físicos e de interesse da humanidade.